In [1]:
import os
from dotenv import load_dotenv
import numpy as np
import google.generativeai as generativeai
from google import genai
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
import pinecone 
import textwrap

c:\Users\LENOVO\AppData\Local\Programs\Python\Python311\Lib\site-packages\pydantic\_internal\_generate_schema.py:502: UserWarning: <built-in function any> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


In [3]:
load_dotenv()
os.environ["GEMINI_API_KEY"] = os.getenv("GEMINI_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["PINECONE_API_KEY"] = os.getenv("PINECONE_API_KEY")

In [4]:
filepath = 'bose_qc_ultra.pdf'
loader = PyPDFLoader(filepath)
pdf_doc = loader.load()
pdf_doc

[Document(metadata={'producer': 'Adobe PDF Library 17.0', 'creator': 'Adobe InDesign 19.3 (Windows)', 'creationdate': '2024-06-07T16:09:58+00:00', 'moddate': '2024-11-19T19:58:18+00:00', 'title': 'English', 'trapped': '/false', 'source': 'bose_qc_ultra.pdf', 'total_pages': 57, 'page': 0, 'page_label': '1'}, page_content='QUIETCOMFORT  ULTRA HEADPHONES\nC ue them usic.'),
 Document(metadata={'producer': 'Adobe PDF Library 17.0', 'creator': 'Adobe InDesign 19.3 (Windows)', 'creationdate': '2024-06-07T16:09:58+00:00', 'moddate': '2024-11-19T19:58:18+00:00', 'title': 'English', 'trapped': '/false', 'source': 'bose_qc_ultra.pdf', 'total_pages': 57, 'page': 1, 'page_label': '2'}, page_content='2 |  ENG\nIMPORTANT SAFETY INSTRUCTIONS\nPlease read and keep all safety and use instructions.\nBose Corporation hereby declares that this product is in compliance with the essential requirements \nand other relevant provisions of Directive 2014/53/EU and all other applicable EU directive  \nrequiremen

In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=10)
pdf_chunks = text_splitter.split_documents(pdf_doc)

chunk_data = []
for chunk in pdf_chunks:
    chunk_data.append(chunk.page_content)

chunk_data

['QUIETCOMFORT  ULTRA HEADPHONES\nC ue them usic.',
 '2 |  ENG\nIMPORTANT SAFETY INSTRUCTIONS\nPlease read and keep all safety and use instructions.\nBose Corporation hereby declares that this product is in compliance with the essential requirements \nand other relevant provisions of Directive 2014/53/EU and all other applicable EU directive  \nrequirements. The complete declaration of conformity can be found at: www.Bose.com/compliancewww.Bose.com/compliance',
 'This product conforms to all applicable Electromagnetic Compatibility Regulations 2016 and all other \napplicable UK regulations. The complete declaration of conformity can be found at:  \nwww.Bose.com/compliancewww.Bose.com/compliance\nBose Corporation hereby declares that this product is in compliance with the essential requirements \nper Radio Equipment Regulations 2017 and all other applicable UK regulations. The complete',
 'declaration of conformity can be found at: www.Bose.com/compliancewww.Bose.com/compliance\nImporta

In [6]:
client = genai.Client(api_key=os.environ["GEMINI_API_KEY"])

In [7]:
embeddings = []
generativeai.configure(api_key=os.environ["GEMINI_API_KEY"])

for chunk in chunk_data:
    result = generativeai.embed_content(model="models/text-embedding-004",content=chunk)
    embeddings.append({'text':chunk, 'vector':result['embedding']})

In [9]:
print(len(embeddings))
print(len(embeddings[0]['vector']))
print(len(chunk_data))

178
768
178


In [ ]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])

pc.create_index(
  name="project-index",
  dimension=768,
  metric="cosine",
  spec=ServerlessSpec(
    cloud="aws",
    region="us-east-1"
  ),
  deletion_protection="disabled"
)

In [11]:
index = pc.Index("project-index")

records = []
id=0
for embedding in embeddings:
    records.append({
        "id": str(id),
        "values": embedding['vector'],
        "metadata":{'text':embedding['text']}
    })
    id = id+1

index.upsert(
    vectors=records,
    namespace="ProjectVectorStore"
)

upserted_count: 178

In [14]:
query = "Tell about activesense feature"
query_embedding =   generativeai.embed_content(model="models/text-embedding-004",content=query)['embedding']
query_result = index.query(
    vector=query_embedding,  
    top_k=5,  
    namespace="ProjectVectorStore",
    include_metadata=True
)


context = ""
for match in query_result['matches']:
    information = match['metadata']['text']
    context = context + information + '\n\n'

prompt = ""
prompt = "The user will ask some question. The context of the information will be provided. Answer from the given context only. DO NOT ANSWER FROM YOUR KNOWLEDGE OR TRY TO MAKE UP SOME ANSWER. IF ANSWER IS NOT PRESENT REPLY 'I DO NOT KNOW'"

final_query = prompt + '\n\n' + context +'\n\n' + query


response = client.models.generate_content(
    model="gemini-1.5-flash", contents=final_query
)

answer = textwrap.fill(response.text,width=100)
print(answer)

Aware mode with ActiveSense technology allows you to hear your surroundings while reducing unwanted
noise.  It automatically increases noise cancellation when sudden or loud noises occur, and then
automatically returns to the previous setting once the noise stops.  To enable ActiveSense, use the
Bose app; tap Modes > Aware >  (the instruction cuts off at this point in the provided text).


In [15]:
ans = textwrap.fill(context,width=100)
print(ans)

T o enable ActiveSense, use the Bose app. T o access this option, on the main screen,  tap Modes >
Aware >  .  LISTENING MODES Modes...................................................................
....................................................................................................
............  25 Aware mode with ActiveSense .......................................................
................................................................ 25  The dynamic noise cancellation
of Aware mode with ActiveSense technology allows  you to hear your surroundings while reducing
unwanted noise. Aware mode with ActiveSense allows the headphones to automatically increase noise
cancellation when sudden or loud noises occur near you, so you can maintain   awareness without
intermittent noise interrupting your audio. Once the noise stops,  noise cancellation automatically
returns the previous setting.  NOTE: You can create up to seven custom modes using the Bose app.   T
o access